# Mounting Goolge drive.

First add the data to your google drive using the following link:

https://drive.google.com/drive/u/1/folders/17SpWmNSl9dcbyqUFpdvnYlHj1EYXwHju

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installing Libraries


In [ ]:
!pip3 install transformers
!pip3 install unidecode

     |████████████████████████████████| 1.3MB 5.9MB/s 
     |████████████████████████████████| 1.1MB 39.1MB/s 
     |████████████████████████████████| 2.9MB 37.8MB/s 
     |████████████████████████████████| 890kB 44.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=98ae5f50c5f318473eb31adc6dbc75db9d0665de3c757ab949a65bc4575a5b89
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 245kB 4.3MB/s 


In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=0453e034890d6c1447b32554a658b3ea87c13badff3a10917bea3e5c7d05cda0
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  |     Proc size: 112.0 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total     16280MB


In [ ]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import join
import unidecode
import re
import logging
from tqdm.notebook import tnrange
import glob
import json

#For ploting results
import matplotlib.pyplot as plt

# DL Libraries
from transformers import BertModel, AdamW, BertTokenizer, BertConfig, RobertaTokenizer, RobertaModel
from keras.preprocessing.sequence import pad_sequences
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("device: {} n_gpu: {}".format(device, n_gpu)) 

device: cuda n_gpu: 1


In [ ]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)
print(logger)

<Logger __main__ (INFO)>


# Loading Dev data set

In [ ]:
!pip install jsonlines
import jsonlines
import pandas as pd
id=[]  
question=[]   
label_list=[] 
A_choice=[] 
B_choice=[] 
C_choice=[] 
D_choice=[] 
Actual_Answer=[] 
fact_1=[] 

with jsonlines.open('/content/drive/MyDrive/NLP/dev_complete.jsonl') as f:
    for line in f.iter():
      question.append(line['question']['stem'])
      label_list.append(line['question']['choices'])
      id.append(line["id"])
      Actual_Answer.append(line['answerKey'])
      fact_1.append(line['fact1'])

for i in range(0,len(label_list)):
  for j in range(0,4):
    if(label_list[i][j]['label']=="A"):
      A_choice.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="B"):
      B_choice.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="C"):
      C_choice.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="D"):
      D_choice.append(label_list[i][j]['text'])    
  
merged_list = tuple(zip(id,question,Actual_Answer,A_choice,B_choice,C_choice,D_choice,fact_1))
data=pd.DataFrame(merged_list,columns=['ID','Question','Actual Answer','A','B','C','D','Fact 1'])
data

,ID,Question,Actual Answer,A,B,C,D,Fact 1
0,8-376,Frilled sharks and angler fish live far beneat...,A,Deep sea animals,fish,Long Sea Fish,Far Sea Animals,deep sea animals live deep in the ocean
1,7-57,"Gas can fill any container it is given, and li...",D,is standard weight and size,is the opposite of variable,only needs a few,uses what it needs,Matter in the liquid phase has definite volume
2,7-1024,"When birds migrate south for the winter, they ...",A,they are genetically called to,their children ask for them to,it is important to their happiness,they decide to each year,migration is an instinctive behavior
3,959,If a person walks in the opposite direction of...,D,west,north,east,south,a compass is a kind of tool for determining di...
4,9-241,An example of lots kinetic energy would be,D,Drinking a cold glass of water,A snail moving across the sidewalk,sitting without moving anywhere,An aircraft taking a trip,"as an object moves , the kinetic energy of tha..."
...,...,...,...,...,...,...,...,...
495,7-640,"If a tree burns down, it can be made right by",D,buying a plastic tree,visiting a rain forest,reading about it in the paper,placing seed in dirt,a tree can be replaced by planting a new tree
496,1374,What role does the first step in the food chai...,D,secondary consumer,decomposer,consumer,producer,In the food chain process a green plant has th...
497,1151,Water at 50 degree C is,A,equally distant from the melting and boiling p...,closer to the boiling point of water,closer to the melting point of ice,below the melting point of ice,melting point means temperature at which a sol...
498,9-469,A phase change is,D,Growing a plant,Building a car,Water changing temperatures,Jello's transformation,temperature changing can cause phase changes


In [ ]:
with open("/content/drive/MyDrive/NLP/crowdsourced-facts.txt", "a") as myfile:
  for row in fact_1:
    myfile.write(row)
    myfile.write("\n")

In [ ]:
merged_list = tuple(zip(question,fact_1))
data_df=pd.DataFrame(merged_list,columns=['question','exact_answer'])
data_df

,question,exact_answer
0,Frilled sharks and angler fish live far beneat...,deep sea animals live deep in the ocean
1,"Gas can fill any container it is given, and li...",Matter in the liquid phase has definite volume
2,"When birds migrate south for the winter, they ...",migration is an instinctive behavior
3,If a person walks in the opposite direction of...,a compass is a kind of tool for determining di...
4,An example of lots kinetic energy would be,"as an object moves , the kinetic energy of tha..."
...,...,...
495,"If a tree burns down, it can be made right by",a tree can be replaced by planting a new tree
496,What role does the first step in the food chai...,In the food chain process a green plant has th...
497,Water at 50 degree C is,melting point means temperature at which a sol...
498,A phase change is,temperature changing can cause phase changes


# Data Loader


In [ ]:
def create_dataloader(tokenizer, df):
    input_ids= list()
    attention_masks= list()

    special_sentences_1 = [sentence for i, sentence in enumerate(df.question)]
    special_sentences_2 = [" [SEP] " + str(sentence) for i, sentence in enumerate(df.answer)]
    special_sentences = [i + j for i, j in zip(special_sentences_1, special_sentences_2)]

    for sentence in special_sentences:
      encoded_text = tokenizer.encode_plus(sentence, max_length=512, add_special_tokens=True, return_token_type_ids=False, 
                                       padding='max_length', return_attention_mask=True, truncation=True)
      input_ids.append(encoded_text['input_ids'])
      attention_masks.append(encoded_text['attention_mask'])

    inputs = torch.tensor(input_ids).to(device)
    masks = torch.tensor(attention_masks).to(device)
    data = TensorDataset(inputs, masks)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=1)

    return dataloader

# Computing 50 sentences for each question from BM25

In [ ]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi
corpus = []
with open('/content/drive/MyDrive/NLP/crowdsourced-facts1.txt', 'r') as f:
    corpus = f.readlines()
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
for i in range(0,len(corpus)):
  corpus[i] = corpus[i].rstrip("\n")

In [ ]:
def get_top50_passages(query):
  tokenized_query = query.split(" ")
  doc_scores = bm25.get_scores(tokenized_query)
  l = bm25.get_top_n(tokenized_query, corpus, n=50)
  return l

In [ ]:
merged_list = tuple(zip(question,fact_1))
data_df1=pd.DataFrame(merged_list,columns=['question','exact_answer'])
data_df1

,question,exact_answer
0,Frilled sharks and angler fish live far beneat...,deep sea animals live deep in the ocean
1,"Gas can fill any container it is given, and li...",Matter in the liquid phase has definite volume
2,"When birds migrate south for the winter, they ...",migration is an instinctive behavior
3,If a person walks in the opposite direction of...,a compass is a kind of tool for determining di...
4,An example of lots kinetic energy would be,"as an object moves , the kinetic energy of tha..."
...,...,...
495,"If a tree burns down, it can be made right by",a tree can be replaced by planting a new tree
496,What role does the first step in the food chai...,In the food chain process a green plant has th...
497,Water at 50 degree C is,melting point means temperature at which a sol...
498,A phase change is,temperature changing can cause phase changes


In [ ]:
finallist = []
for i in range(0,len(question)):
  question_list=[]
  query = question[i]
  tokenized_query = query.split(" ")
  doc_scores = bm25.get_scores(tokenized_query)
  l = bm25.get_top_n(tokenized_query, corpus, n=50)
  question_list.append(l)
  finallist.append(question_list)

In [ ]:
bm25_list_questions = []
bm25_list_answers =[]
for i in range(0,500):
  for j in range(0,50):
    bm25_list_questions.append(question[i])
    bm25_list_answers.append(finallist[i][0][j])

# Model


In [ ]:
class Regressor(nn.Module):

  def __init__(self, model_path):
    super(Regressor, self).__init__()
    self.bert = RobertaModel.from_pretrained(model_path)
    self.out = nn.Linear(self.bert.config.hidden_size, 1)

  def forward(self, input_ids, attention_mask):
    output, pooler_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    score= self.out(pooler_out)
    return score

In [ ]:
model_path= '/content/drive/MyDrive/NLP/sia_experiment/SIA_OpenBookQA'

#Load Model
model= Regressor(model_path)
lr_weights= torch.load(join(model_path, 'model_state.bin'))
model.out.load_state_dict(lr_weights)
model.to(device)


tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

11/29/2020 19:38:10 - INFO - filelock -   Lock 140541457299216 acquired on /root/.cache/torch/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock


11/29/2020 19:38:11 - INFO - filelock -   Lock 140541457299216 released on /root/.cache/torch/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock
11/29/2020 19:38:11 - INFO - filelock -   Lock 140541457263584 acquired on /root/.cache/torch/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


11/29/2020 19:38:11 - INFO - filelock -   Lock 140541457263584 released on /root/.cache/torch/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


# Pyserini

In [ ]:
!pip install pyserini==0.9.4.0

In [ ]:
%cd /content/drive/My Drive/NLP/corpus

/content/drive/My Drive/NLP/corpus


In [ ]:
import os       
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"     
  !java -version     
install_java()

openjdk version "11.0.9.1" 2020-11-04
OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


In [ ]:
!git clone https://github.com/castorini/pyserini.git

fatal: destination path 'pyserini' already exists and is not an empty directory.


In [ ]:
sentences= []
index=[]
i=1
corpus_sentences=open('/content/drive/MyDrive/NLP/crowdsourced-facts1.txt','r')
for line in corpus_sentences.readlines():
    lin = line.replace('\n', ' ')
    index.append(i)
    sentences.append(lin)
    i=i+1

In [ ]:
%cd /content/drive/My Drive/NLP/corpus

/content/drive/My Drive/NLP/corpus


In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(index, sentences)), columns=['index','paragraph'])
df.to_csv('/content/drive/My Drive/NLP/corpus/pyserini/collections/document_passages.tsv', header=False, index=False, sep='\t')

In [ ]:
df_data= pd.read_csv('/content/drive/My Drive/NLP/corpus/pyserini/collections/document_passages.tsv', sep='\t')

In [ ]:
!python pyserini/anserini-tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path pyserini/collections/document_passages.tsv \
 --output-folder pyserini/collections/collection_jsonl

Converting collection...
Converted 0 docs, writing into file 1
Done!


In [ ]:
with open('/content/drive/My Drive/NLP/corpus/pyserini/collections/document_passages.tsv', encoding='utf-8') as f:
  for i, line in enumerate(f):
      doc_id,doc_text= line.split("\t")
      doc_text = doc_text.rstrip()
      print(doc_id + " " + doc_text)

Streaming output truncated to the last 5000 lines.
668 Puppigerus was a sea turtle
669 Putting a flower in food dye allows you to see how the xylem works.
670 Putting plastic over a plant speeds up the process of evaporation.
671 Putting something cool on your hot body will cool you down
672 Quartz are in hollow spaces.
673 RVs are made from iron.
674 Rabbits tunnel underground.
675 Raccoons are animals
676 Raccoons can see in the dark
677 Racquetballs are fast
678 Rain also occurs in other places like planetoids.
679 Rain helps communities thrive.
680 Recorders are sometimes electromagnets.
681 Recycling is another form of conservation
682 Red blood cells help your breathing boxes
683 Redeeming aluminum cans at a store or recyling center is one example of this
684 Redwood National Park is a forest of giant trees
685 Redwoods are a type of tree that could be found in a forest
686 Redwoods are trees
687 Reflection is due to how difficult it is for light to pass through it.
688 Reflector

In [ ]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 9 -input pyserini/collections/collection_jsonl \
 -index pyserini/indexes/lucene-index-open -storePositions -storeDocvectors -storeRaw

2020-11-29 19:38:25,582 INFO  [main] index.IndexCollection (IndexCollection.java:636) - Setting log level to INFO
2020-11-29 19:38:25,585 INFO  [main] index.IndexCollection (IndexCollection.java:639) - Starting indexer...
2020-11-29 19:38:25,585 INFO  [main] index.IndexCollection (IndexCollection.java:640) - ============ Loading Parameters ============
2020-11-29 19:38:25,586 INFO  [main] index.IndexCollection (IndexCollection.java:641) - DocumentCollection path: pyserini/collections/collection_jsonl
2020-11-29 19:38:25,586 INFO  [main] index.IndexCollection (IndexCollection.java:642) - CollectionClass: JsonCollection
2020-11-29 19:38:25,587 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Generator: DefaultLuceneDocumentGenerator
2020-11-29 19:38:25,587 INFO  [main] index.IndexCollection (IndexCollection.java:644) - Threads: 9
2020-11-29 19:38:25,588 INFO  [main] index.IndexCollection (IndexCollection.java:645) - Stemmer: porter
2020-11-29 19:38:25,588 INFO  [main] inde

In [ ]:
from pyserini.search import SimpleSearcher
import pandas as pd
from os.path import join
import numpy as np

searcher = SimpleSearcher('/content/drive/My Drive/NLP/corpus/pyserini/indexes/lucene-index-open')
searcher.set_bm25(0.4, 0.1)
hits = searcher.search("Tunnels", k =50)

# Print the first 10 hits:
for hit in (0,50):
    print(hit)

0
50


In [ ]:
R= []
for i in range(0,len(hits)):
  doc = searcher.doc(str(hits[i].docid))
  R.append(doc.raw())

# Evaluation

In [ ]:
def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

In [ ]:
def recall_at_k(actual, predicted, k):
  count=0
  predicted= predicted[0:k]
  for a in actual:
    if a in predicted:
      count+=1
  return count/len(actual)

In [ ]:
def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

In [ ]:
def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

# BM25 baseline model evaluation

In [ ]:
from statistics import mean 
def get_candidate_passages_with_scores(query):

  candidate_passages=[]
  scores=[]
  hits = searcher.search(query, k=50)
  # return the first top 10 hits:
  for hit in hits:
    doc = searcher.doc(str(hit.docid))
    candidate_passages.append(doc.raw().replace('"', ''))
    scores.append(hit.score)
  
  return candidate_passages,scores

In [ ]:
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_combined_fact=0
for index, row in data_df.iterrows():
  if(index == 18 or index == 31 or index==129 or index == 180 or index==300 or index == 310 or index == 311 or index == 462 or index== 544 
     or index == 712):
    continue
  print(index)
  answers,score= get_candidate_passages_with_scores(row.question)
  temp_df= pd.DataFrame(list(zip(answers,score)), columns=['answer','score'])
  temp_df['question']=row.question
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()
  org_ans=[]
  org_ans.append(row.exact_answer)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.exact_answer:
      contains_combined_fact+=1
  
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))

print("Precision @5: ",mean(precision_at_5))
print("Precision @10: ",mean(precision_at_10))
print("Recall @5: ",mean(recall_at_5))
print("Recall @10: ",mean(recall_at_10))
print("Recall @20: ",mean(recall_at_20))
print("Mean Average Precision: ",mean_average_precision(rs))
print("Mean Reciprocal Rank: ",mean_reciprocal_rank(rs))
print("contains_fact",contains_combined_fact)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
19
20
21
22
23
24
25
26
27
28
29
30
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280


# SIA Model 

In [ ]:
def get_candidate_passages(query):

  candidate_passages=[]
  hits = searcher.search(query, k=50)
  # return the first top 10 hits:
  for hit in hits:
    doc = searcher.doc(str(hit.docid))
    candidate_passages.append(doc.raw().replace('"', ''))
  
  return candidate_passages

In [ ]:
from statistics import mean 
import pandas as pd
precision_at_5=[]
precision_at_10=[]
recall_at_5=[]
recall_at_10=[]
recall_at_20=[]
rs=[]
contains_combined_fact=0
for index, row in data_df.iterrows():
  if(index == 18 or index == 31 or index==129 or index == 180 or index==300 or index == 310 or index == 311 or index == 462):
    continue
  answers= get_candidate_passages(row.question)
  temp_df= pd.DataFrame(answers, columns=['answer'])
  temp_df['question']=row.question

  dataloader= create_dataloader(tokenizer, temp_df)
  pos_prob= []
  model.eval()
  with torch.no_grad():
    for step, batch in enumerate(dataloader):
      batch = tuple(t.to(device) for t in batch)
      ip_ids, masks= batch
      score = model(ip_ids, attention_mask=masks)
      pos_prob.append(score.cpu().detach().numpy())
  
  temp_df['score']=pos_prob
  temp_df= temp_df.sort_values(by='score', ascending=False)
  top_k_ans= temp_df[0:20].answer.tolist()
  org_ans=[]
  # for no in passages_no:
  org_ans.append(row.exact_answer)
  r=[]
  for ans in top_k_ans:
    if ans in org_ans:
      r.append(1)
    else:
      r.append(0)
  for ans in top_k_ans:
    if ans==row.exact_answer:
      contains_combined_fact+=1
  print(index)
  rs.append(r)
  precision_at_5.append(precision_at_k(r,5))
  precision_at_10.append(precision_at_k(r,10))
  recall_at_5.append(recall_at_k(org_ans, top_k_ans, 5))
  recall_at_10.append(recall_at_k(org_ans, top_k_ans, 10))
  recall_at_20.append(recall_at_k(org_ans, top_k_ans, 20))
print("Precision @5: ",mean(precision_at_5))
print("Precision @10: ",mean(precision_at_10))
print("Recall @5: ",mean(recall_at_5))
print("Recall @10: ",mean(recall_at_10))
print("Recall @20: ",mean(recall_at_20))
print("Mean Average Precision: ",mean_average_precision(rs))
print("Mean Reciprocal Rank: ",mean_reciprocal_rank(rs))
print("contains_fact",contains_combined_fact)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
19
20
21
22
23
24
25
26
27
28
29
30
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
